In [ ]:
import numpy as np

In [ ]:
# ok = np.load('./ok.npy')

stop = np.load('./stop.npy')
neutral = np.load('./neutral.npy')
right = np.load('./right.npy')
left = np.load('./left.npy')

In [ ]:
print(stop.shape)
print(neutral.shape)
print(right.shape)
print(left.shape)

(1578, 42)
(1021, 42)
(1471, 42)
(1307, 42)


In [ ]:
def normalize_keypoints(kp):
  for i in range(0, len(kp)):
      non_zero_x = kp[i][0:21][kp[i][0:21] != 0]
      non_zero_y = kp[i][21:42][kp[i][21:42] != 0]

      min_val = [np.min(non_zero_x), np.min(non_zero_y)]
      max_val = [np.max(non_zero_x), np.max(non_zero_y)]

      kp[i][0:21] = (kp[i][0:21] - min_val[0]) / (max_val[0] - min_val[0])

      kp[i][21:42] = (kp[i][21:42] - min_val[1]) / (max_val[1] - min_val[1])
  return kp

In [ ]:
stop = normalize_keypoints(stop)
neutral = normalize_keypoints(neutral)
right = normalize_keypoints(right)
left = normalize_keypoints(left)

In [ ]:
import matplotlib.pyplot as plt
def poselines2(frame):
  # plt.xlim(0, 1)  # Set limits for x-axis
  # plt.ylim(-1, 0) # Set limits for y-axis
  x = frame[0:21]
  y = frame[21:42]
  xbl = [0,4,0,6,3,9,11,11,12,13,14]
  ybl = [4,7,4,8,7,10,13,12,14,15,16]
  xh = [17, 22, 26, 30, 34, 17, 22, 26, 30, 17]
  yh = [22, 26, 30, 34, 38, 22, 26, 30, 34, 34]

  for i in range(21):
    if (x[i] != 0 ):
      plt.plot(x[i], -y[i], 'bo-')

In [ ]:
# for i in ok:
#   poselines2(i)
#   plt.show()

In [ ]:
# ok = ok[:1000, :]
stop = stop[:1000, :]
neutral = neutral[:1000, :]
right = right[:1000, :]
left = left[:1000, :]

In [ ]:
stop.shape

(1000, 42)

In [ ]:
X = np.concatenate([stop, neutral, right, left])

In [ ]:
X.shape

(4000, 42)

In [ ]:
num_labels = X.shape[0] // 1000
labels = np.repeat(np.arange(num_labels), 1000)
labels = labels
labels.shape

(4000,)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels).astype(int)
y.shape

(4000, 4)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=0)
test_data.shape

(800, 42)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Assume X is your input data of shape (N samples, K keypoints) and y is your labels
# X = np.array(...)  # shape (N, K)
# y = np.array(...)  # shape (N, num_classes)

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

# # Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(train_data, train_labels, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {accuracy:.2f}')

Epoch 1/50
80/80 [==============================] - 2s 6ms/step - loss: 0.9611 - accuracy: 0.6020 - val_loss: 0.3438 - val_accuracy: 0.8906
Epoch 2/50
80/80 [==============================] - 0s 4ms/step - loss: 0.3594 - accuracy: 0.8742 - val_loss: 0.0852 - val_accuracy: 0.9828
Epoch 3/50
80/80 [==============================] - 0s 4ms/step - loss: 0.1917 - accuracy: 0.9395 - val_loss: 0.0323 - val_accuracy: 0.9906
Epoch 4/50
80/80 [==============================] - 0s 4ms/step - loss: 0.1117 - accuracy: 0.9688 - val_loss: 0.0177 - val_accuracy: 0.9922
Epoch 5/50
80/80 [==============================] - 0s 4ms/step - loss: 0.0764 - accuracy: 0.9777 - val_loss: 0.0112 - val_accuracy: 0.9953
Epoch 6/50
80/80 [==============================] - 0s 4ms/step - loss: 0.0513 - accuracy: 0.9852 - val_loss: 0.0079 - val_accuracy: 0.9984
Epoch 7/50
80/80 [==============================] - 0s 4ms/step - loss: 0.0490 - accuracy: 0.9836 - val_loss: 0.0062 - val_accuracy: 0.9984
Epoch 8/50
80/80 [==

In [ ]:
model.save_weights("./weights4c.h5")

In [ ]:
model.load_weights('./weights4c.h5')

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {accuracy:.2f}')

25/25 [==============================] - 0s 3ms/step - loss: 3.0385e-04 - accuracy: 1.0000
Test accuracy: 1.00


In [ ]:
key = X[0].reshape(1, -1)
key.shape

(1, 42)

In [ ]:
sign = model.predict(key)
sign

1/1 [==============================] - 0s 101ms/step


array([[9.9999976e-01, 2.6300603e-07, 4.3536681e-09, 3.8784491e-09]],
      dtype=float32)

In [ ]:
from graphviz import Digraph

dot = Digraph()
dot.node('Entrada', 'Capa de Entrada\n42 unidades')
dot.node('Densa1', 'Capa Densa\n128 unidades\nActivación ReLU')
dot.node('Dropout1', 'Capa Dropout\n0.5')
dot.node('Densa2', 'Capa Densa\n64 unidades\nActivación ReLU')
dot.node('Dropout2', 'Capa Dropout\n0.5')
dot.node('Densa3', 'Capa Densa\n32 unidades\nActivación ReLU')
dot.node('Densa4', 'Capa Densa\n4 unidades\nActivación Softmax')

dot.edge('Entrada', 'Densa1')
dot.edge('Densa1', 'Dropout1')
dot.edge('Dropout1', 'Densa2')
dot.edge('Densa2', 'Dropout2')
dot.edge('Dropout2', 'Densa3')
dot.edge('Densa3', 'Densa4')

dot.render('arquitectura_red_neuronal', format='png')

'arquitectura_red_neuronal.png'